In [3]:
import requests
from bs4 import BeautifulSoup
import os
import time
from keras.utils import get_file
try:
    from urllib.request import urlretrieve
except ImportError:
    from urllib import urlretrieve
import xml.sax

import subprocess
import mwparserfromhell
import json

In [4]:
index = requests.get('https://dumps.wikimedia.org/enwiki/').text

In [6]:
index  # WS 11/13/18

'<html>\r\n<head><title>Index of /enwiki/</title></head>\r\n<body bgcolor="white">\r\n<h1>Index of /enwiki/</h1><hr><pre><a href="../">../</a>\r\n<a href="20180801/">20180801/</a>                                          21-Sep-2018 01:33                   -\r\n<a href="20180820/">20180820/</a>                                          02-Oct-2018 01:27                   -\r\n<a href="20180901/">20180901/</a>                                          21-Oct-2018 01:31                   -\r\n<a href="20180920/">20180920/</a>                                          02-Nov-2018 01:27                   -\r\n<a href="20181001/">20181001/</a>                                          11-Oct-2018 17:43                   -\r\n<a href="20181020/">20181020/</a>                                          26-Oct-2018 05:45                   -\r\n<a href="20181101/">20181101/</a>                                          11-Nov-2018 11:16                   -\r\n<a href="latest/">latest/</a>             

In [7]:
soup_index = BeautifulSoup(index, 'html.parser')

In [9]:
soup_index  # WS 11/13/18

<html>
<head><title>Index of /enwiki/</title></head>
<body bgcolor="white">
<h1>Index of /enwiki/</h1><hr/><pre><a href="../">../</a>
<a href="20180801/">20180801/</a>                                          21-Sep-2018 01:33                   -
<a href="20180820/">20180820/</a>                                          02-Oct-2018 01:27                   -
<a href="20180901/">20180901/</a>                                          21-Oct-2018 01:31                   -
<a href="20180920/">20180920/</a>                                          02-Nov-2018 01:27                   -
<a href="20181001/">20181001/</a>                                          11-Oct-2018 17:43                   -
<a href="20181020/">20181020/</a>                                          26-Oct-2018 05:45                   -
<a href="20181101/">20181101/</a>                                          11-Nov-2018 11:16                   -
<a href="latest/">latest/</a>                                            11

In [12]:
soup_index.find_all('a')  # WS 11/13/18

[<a href="../">../</a>,
 <a href="20180801/">20180801/</a>,
 <a href="20180820/">20180820/</a>,
 <a href="20180901/">20180901/</a>,
 <a href="20180920/">20180920/</a>,
 <a href="20181001/">20181001/</a>,
 <a href="20181020/">20181020/</a>,
 <a href="20181101/">20181101/</a>,
 <a href="latest/">latest/</a>]

In [20]:
dd = soup_index.find_all('a')[1]
type(dd)  # WS 11/13/18

bs4.element.Tag

In [22]:
dd['href']  # WS 11/13/18

'20180801/'

In [25]:
dd.text[:-1]  # WS 11/13/18  all chars except the last one

'20180801'

In [10]:
dumps = [a['href'] for a in soup_index.find_all('a') 
             if a.has_attr('href') and a.text[:-1].isdigit()]
dumps

['20180801/',
 '20180820/',
 '20180901/',
 '20180920/',
 '20181001/',
 '20181020/',
 '20181101/']

In [26]:
for dump_url in sorted(dumps, reverse=True):
    print(dump_url)
    dump_html = index = requests.get('https://dumps.wikimedia.org/enwiki/' + dump_url).text
    soup_dump = BeautifulSoup(dump_html, 'html.parser')
    pages_xml = [a['href'] for a in soup_dump.find_all('a') 
                 if a.has_attr('href') and a['href'].endswith('-pages-articles.xml.bz2')]
    if pages_xml:
        break
    time.sleep(0.8)

20181101/


In [32]:
pages_xml[0]  # WS 11/13/18  the file below is 14.5 GB!

'/enwiki/20181101/enwiki-20181101-pages-articles.xml.bz2'

In [31]:
wikipedia_dump = pages_xml[0].rsplit('/')[-1]
wikipedia_dump  # WS 11/13/18

'enwiki-20181101-pages-articles.xml.bz2'

## WS 11/13/18  Stopped short here, before downloading the 14.5 GB file!!!

In [ ]:
url = url = 'https://dumps.wikimedia.org/' + pages_xml[0] 
path = get_file(wikipedia_dump, url)
path

 7939751936/15398410099 [==============>...............] - ETA: 1:00:18

In [ ]:
def process_article(title, text):
    rotten = [(re.findall('\d\d?\d?%', p), re.findall('\d\.\d\/\d+|$', p), p.lower().find('rotten tomatoes')) for p in text.split('\n\n')]
    rating = next(((perc[0], rating[0]) for perc, rating, idx in rotten if len(perc) == 1 and idx > -1), (None, None))
    wikicode = mwparserfromhell.parse(text)
    film = next((template for template in wikicode.filter_templates() 
                 if template.name.strip().lower() == 'infobox film'), None)
    if film:
        properties = {param.name.strip_code().strip(): param.value.strip_code().strip() 
                      for param in film.params
                      if param.value.strip_code().strip()
                     }
        links = [x.title.strip_code().strip() for x in wikicode.filter_wikilinks()]
        return (title, properties, links) + rating

In [ ]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._movies = []
        self._curent_tag = None

    def characters(self, content):
        if self._curent_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        if name in ('title', 'text'):
            self._curent_tag = name
            self._buffer = []

    def endElement(self, name):
        if name == self._curent_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            movie = process_article(**self._values)
            if movie:
                self._movies.append(movie)

In [ ]:
parser = xml.sax.make_parser()
handler = WikiXmlHandler()
parser.setContentHandler(handler)
for line in subprocess.Popen(['bzcat'], stdin=open(path), stdout=subprocess.PIPE).stdout:
    try:
        parser.feed(line)
    except StopIteration:
        break

In [ ]:
with open('generated/wp_movies.ndjson', 'wt') as fout:
    for movie in handler._movies:
         fout.write(json.dumps(movie) + '\n')